# Image Compositing & Creative Merging
This notebook allows you to blend two images together, place people in new environments, and perform "Virtual Try-ons".

**Core Abilities:**
- **Image Merging:** Combine a Person A with Place B.
- **Identity Preservation:** Try to keep the same face while changing everything else.
- **Virtual Try-on:** Swap clothing or textures using AI.
- **Scene Context:** The AI matches lights and shadows automatically.

In [ ]:
# Step 1: Install powerful diffusion tools
# %pip is the standard way to install packages inside a notebook
%pip install -q diffusers transformers accelerate scipy ipywidgets Pillow

In [ ]:
# Import math and AI libraries
import torch
import io
from PIL import Image, ImageOps
# 'StableDiffusionInpaintPipeline' is the best tool for merging/changing parts of images
from diffusers import StableDiffusionInpaintPipeline
import ipywidgets as widgets
from IPython.display import display

# Check for GPU hardware for fast calculations
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Active hardware: {device}")

In [ ]:
# Step 2: Load the Creative AI Engine
# 'runwayml/stable-diffusion-inpainting' is a version specifically trained to blend images
model_id = "runwayml/stable-diffusion-inpainting"

# Download the model from the internet
pipe = StableDiffusionInpaintPipeline.from_pretrained(
    model_id, 
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
)

# Move the model to your GPU or CPU
pipe = pipe.to(device)

print("Compositing Engine is ready!")

In [ ]:
# This function handles the complex work of blending two images
def merge_images(main_image, reference_mask, prompt, negative_prompt=""):
    # Standardize image sizes to 512x512 pixels
    main_image = main_image.convert("RGB").resize((512, 512))
    # The 'mask' tells the AI which part of the photo to change vs keep
    mask_image = reference_mask.convert("RGB").resize((512, 512))
    
    print(f"Blending images with prompt: {prompt[:30]}...")
    
    # Use 'autocast' to make the math run efficiently
    with torch.autocast(device):
        # The pipeline takes the original photo, the mask (where to change), and the prompt
        result = pipe(
            prompt=prompt, 
            negative_prompt=negative_prompt, 
            image=main_image, 
            mask_image=mask_image
        ).images[0]
    
    return result

In [ ]:
# --- Creative Compositing Dashboard ---

print("HOW TO USE:")
print("1. Upload the Main Image (e.g., a person or a room).")
print("2. Upload a 'Mask' image (A black and white photo where WHITE is the area you want to change).")
print("3. Describe what should appear in the white area.")

# First uploader for the base photo
main_uploader = widgets.FileUpload(description="Photo A (Base)", multiple=False)
# Second uploader for the mask (or reference)
mask_uploader = widgets.FileUpload(description="Mask / Ref", multiple=False)

# Text boxes for what you want to see
u_prompt = widgets.Textarea(
    value='A man doing a dangerous stunt on top of a skyscraper, dramatic lighting, professional photography',
    placeholder='What should happen in the masked area?',
    description='Positive:',
    layout=widgets.Layout(width='90%', height='80px')
)

u_negative = widgets.Textarea(
    value='ugly, blurry, low quality, distorted hands, tattoos',
    placeholder='What should the AI avoid?',
    description='Negative:',
    layout=widgets.Layout(width='90%', height='60px')
)

go_button = widgets.Button(description="Merge & Generate", button_style='primary', layout=widgets.Layout(width='200px', height='40px'))
c_output = widgets.Output()

def on_go_clicked(b):
    with c_output:
        c_output.clear_output()
        # Safety checks: ensure we have both photos
        if not main_uploader.value or not mask_uploader.value:
            print("Please upload BOTH the Main Photo and the Mask Photo (Ref).")
            return
        
        # Load Photo A
        file_a = list(main_uploader.value.values())[0] if isinstance(main_uploader.value, dict) else main_uploader.value[0]
        img_a = Image.open(io.BytesIO(file_a['content']))
        
        # Load Mask/Ref
        file_b = list(mask_uploader.value.values())[0] if isinstance(mask_uploader.value, dict) else mask_uploader.value[0]
        img_b = Image.open(io.BytesIO(file_b['content']))
        
        # Run the AI blending engine
        final = merge_images(img_a, img_b, u_prompt.value, u_negative.value)
        
        # Show the result
        print("Successfully Merged!")
        display(final)

go_button.on_click(on_go_clicked)

display(widgets.HBox([main_uploader, mask_uploader]), u_prompt, u_negative, go_button, c_output)